In [1]:
import os
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.io import mmread, mmwrite
import torch

def save_ilu_matrix(input_mtx_file, output_folder):
    # Load the sparse matrix from the .mtx file
    A = mmread(input_mtx_file).tocsc()  # Ensure the matrix is in CSC format
    
    # Perform ILU factorization
    ilu = spla.spilu(A)
    
    # Extract L and U from the ILU factorization (spilu)
    L = sp.tril(ilu.L, format='csr')  # Lower triangular matrix
    U = sp.triu(ilu.U, format='csr')  # Upper triangular matrix
    
    # Multiply L and U to form the combined LU matrix
    LU = L @ U  # Sparse matrix multiplication to maintain sparsity
    
    # Convert the LU matrix to PyTorch sparse tensor
    coo = LU.tocoo()  # Convert to COO format
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = torch.Size(coo.shape)

    # Create PyTorch sparse tensor
    initial_matrix = torch.sparse_coo_tensor(i, v, shape)

    # Print information about the PyTorch sparse tensor
    print(f"PyTorch sparse tensor shape: {initial_matrix.shape}")
    print(f"Number of non-zero elements: {initial_matrix._nnz()}")
    print(f"Indices: {initial_matrix._indices()}")
    print(f"Values: {initial_matrix._values()}")

    # Construct output filename and save the LU matrix as an .mtx file
    original_filename = os.path.splitext(os.path.basename(input_mtx_file))[0]
    output_file = os.path.join(output_folder, f"{original_filename}_ilu.mtx")
    
    # Save the LU matrix to the specified output folder
    mmwrite(output_file, LU)
    print(f"ILU matrix saved to {output_file}")

# Example usage
# save_ilu_matrix('path/to/input_file.mtx', 'path/to/output_folder')


In [4]:
small_matrices = ['data/LF10/LF10.mtx', 'data/Trefethen_20/Trefethen_20.mtx', 'data/can_24/can_24.mtx']

for i in small_matrices:
    save_ilu_matrix(i, 'data/small_ILU')

PyTorch sparse tensor shape: torch.Size([18, 18])
Number of non-zero elements: 90
Indices: tensor([[ 0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,
          4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  7,  7,
          7,  7,  7,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10,
         10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14,
         14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 17, 17, 17, 17, 17],
        [ 4,  3,  1,  0,  6,  5,  3,  2,  1,  2,  4,  3,  0,  8,  7,  5,  4,  3,
          4,  6,  5,  2,  1, 10,  9,  7,  6,  5,  6,  8,  7,  5,  4,  3, 12, 11,
          9,  8,  7,  8, 10,  9,  7,  6,  5, 14, 13, 11, 10,  9, 10, 12, 11,  8,
          7, 17, 15, 13, 12, 11, 12, 14, 13, 10,  9, 16, 15, 14, 13, 14, 17, 15,
         12, 11, 15, 13,  8,  6,  5,  2,  1,  0, 17, 16, 15, 17, 16, 15, 14, 13]])
Values: tensor([ 4.7715e+02, -8.5888e+04,  1.7178e+05, -4.7715e+02,  4.7715e+02,
        -8.5888